In [ ]:
!pip install opencv-python

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
import os
import glob
import cv2

# Splitting dataset (splitting paths leading to images)

## creating dataset with paths to spectograms

In [ ]:
# specify path to spectogram images
PATH = r"C:\Users\Ilan\Documents\ITC\11.Final_Project\2. Dataset 2\Data\images_original"

In [ ]:
PATH

In [ ]:
genres = os.listdir(PATH)

In [ ]:
# getting path of all file names in directories
files = {}
for genre in genres:
    files[genre] = glob.glob(fr"{PATH}\{genre}\*png")

In [ ]:
# creating dataframe wit paths
df = pd.DataFrame(columns = ['path', 'rgb_data', 'label'])
for genre in genres:
    df_aux = pd.DataFrame(files[genre], columns = ['path'])
    df_aux['label'] = genre
    df = df.append(df_aux, ignore_index=True, sort=False)

# verifying number of rows
df['label'].value_counts()

In [ ]:
#getting rgb data for all images
for index in df.index:
    img = cv2.imread(df['path'].loc[index])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #df['rgb_data'].loc[index] = np.array(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), dtype=np.uint8)
    df['rgb_data'].loc[index] = np.asarray(img).astype('float32')

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# encoding label
le = LabelEncoder()
le.fit(genres)
df['label_encoded'] = le.transform(df['label'])

In [ ]:
df['label_encoded']

## splitting dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#defining X and y
X = [data for data in df['rgb_data']]
X = np.array(X)
y = np.array(df['label_encoded'])

#splitting data and stratifying by y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Modelling

In [ ]:
shape_of_input = df['rgb_data'][0].shape
shape_of_input

In [ ]:
model = keras.Sequential([
    layers.Rescaling(1 / 255, input_shape=shape_of_input),
    
    layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    
    layers.Dropout(rate=0.3),
    
    layers.Dense(len(genres), activation='softmax')
])

In [ ]:
print(f'Model input shape: {model.input_shape}, Model output shape: {model.output_shape}')

In [ ]:
print(model.summary())

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')
model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=100)

In [ ]:
y_train.shape

In [ ]:
y_train